### Import

In [ ]:
import json
import pandas as pd
import math
import os
import xmltodict
import traceback
from slpp import slpp as lua
from compress_lua_table import CompressLuaTable as compress

In [ ]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

### `csv`导出`lua`

In [ ]:
types = {}
_types = {}
heads = set()
# with open('SkillEffectTable.json', 'rb') as f:
#     data = json.load(f)
#     for field in data['Fields']:
#         types[field['FieldName']] = field
#         _types[field['FieldName']] = field['FieldTypeName']
#         heads.add(field['FieldName'])

# set(_types.values())
dir = 'E:\WorkSpace\ROGame-dev\config\Table\Configs'
for file in os.listdir(dir):
    with open(os.path.join(dir, file), 'rb') as file:
        data = json.load(file)
        for item in data['TableLocations']:
            print(item['ExcelPath'])

In [ ]:
data = pd.read_csv('SkillEffectTable.csv').drop([0])
columns = list(set(data.columns.tolist()) - heads)
data = data.drop(columns=columns)
lua_raw_data = data.to_dict('index')

def check_default(_v, cast, default):
    def is_nan(__v):
        if type(__v) == float:
            return math.isnan(__v)
    return default if is_nan(_v) else cast(_v)


def sequence_to_dict(value, partten, length, cast):
    if value is None or value.strip() == '':
        return value
    array = {}
    sequence = value.split(partten)
    if len(sequence) == length:
        for i in range(length):
            array[i + 1] = cast(sequence[i])
        return array
    return value


def vector_to_list(value, partten, cast, func=None, *args):
    if value is None or value.strip() == '':
        return value
    l = []
    sequence = value.split(partten)
    for s in sequence:
        if func is not None:
            l.append(func(s, *args))
        elif s.strip() != '':
            l.append(cast(s))
    return l


def iteritems_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iteritems_recursive(v)
        else:
            _type = _types[k]
            default = types[k]['DefaultValue']
            if _type == 'string':
                t[k] = check_default(v, str, default)
            elif _type == 'int':
                t[k] = check_default(v, int, default)
            elif _type == 'float':
                t[k] = check_default(v, float, default)
            elif _type == 'bool':
                t[k] = 'false' if v == 0 or v == 'nan' else 'true'
            elif _type == 'Sequence<int, 2>':
                t[k] = sequence_to_dict(check_default(v, str, default), '=', 2, int)
            elif _type == 'vector<Sequence<int, 2>>':
                args = ['=', 2, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<Sequence<int, 3>>':
                args = ['=', 3, int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, sequence_to_dict, *args)
            elif _type == 'vector<int>':
                t[k] = vector_to_list(check_default(v, str, default), '|', int)
            elif _type == 'vector<float>':
                t[k] = vector_to_list(check_default(v, str, default), '|', float)
            elif _type == 'vector<string>':
                t[k] = vector_to_list(check_default(v, str, default), '|', str)
            elif _type == 'vector<vector<int>>':
                args = ['=', int]
                t[k] = vector_to_list(check_default(v, str, default), '|', int, vector_to_list, *args)
            else:
                print(f'`{bcolors.FAIL}' + _type + f'{bcolors.RESET}` is not processed!')
                # return
    return t

table = iteritems_recursive(lua_raw_data)
with open('SkillTable.lua', 'w', encoding='utf-8') as f:
    f.write('return ' + lua.encode(table))

In [ ]:
def is_int(n):
    try:
        int(n)
    except ValueError:
        return False
    return True

def is_float(n):
    try:
        float(n)
    except ValueError:
        return False
    return True

def is_boolean(n):
    if n == 'true' or n == 'false':
        return True
    return False

def to_boolean(n):
    if n == 'true': return True
    elif n == 'false': return False

def filter_key(k):
    return k == '@xmlns:xsd' or k == '@xmlns:xsi'

def iter_xml_recursive(d):
    t = {}
    for k, v in d.items():
        if isinstance(v, dict):
            t[k] = iter_xml_recursive(v)
        elif isinstance(v, list):
            t[k] = iter_xml_recursive({i : v[i] for i in range(len(v))})
        else:
            if v is None:
                t[k] = None
            elif is_int(v):
                t[k] = int(v)
            elif is_float(v):
                t[k] = float(v)
            elif is_boolean(v):
                t[k] = to_boolean(v)
            elif not filter_key(k):
                t[k] = v
                # return
    return t

with open('E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData\Activity_XieNengRuQin\Monster_ShenGuan_ShengGuangFaZhen.txt', 'rb') as f:
    table = iter_xml_recursive(dict(xmltodict.parse(f)))
    # print(json.loads(json.dumps(xmltodict.parse(f))))

#### `python`压缩`lua`表

In [ ]:
from compress_lua_table import CompressLuaTable as compress
compress.process_file(table, 'Monster_ShenGuan_ShengGuangFaZhen')

#### `xml`导出`lua`

In [ ]:
dir = 'E:\WorkSpace\ROGame-dev\config\Assets\Resources\SkillData'
EDITOR_DIR = './SkillData/'
if not os.path.exists(EDITOR_DIR):
    os.mkdir(EDITOR_DIR)
for path, dirs, fs in os.walk(dir):
    for f in fs:
        if f.endswith('.txt'):
            # print(f'processing {bcolors.OK}' + os.path.join(path, f) + f'{bcolors.RESET}...')
            # print(os.path.join(EDITOR_DIR + path[len(dir) + 1:], f.replace('.txt', '.lua')))
            _dir = os.path.join(EDITOR_DIR + path[len(dir) + 1:])
            if not os.path.exists(_dir):
                os.mkdir(_dir)
            with open(os.path.join(path, f), 'rb') as file:
                try:
                    table = iter_xml_recursive(dict(xmltodict.parse(file)))
                    with open(os.path.join(_dir, f.replace('.txt', '.lua')), 'w', encoding='utf-8') as w:
                        w.write('local ' + f[:-4] + ' = ' + lua.encode(table) + '\n return ' + f[:-4])
                except Exception as e:
                    print(f'{bcolors.FAIL} error: ' + os.path.join(path, f) + f'{bcolors.RESET}')
                    traceback.print_exc()


> 数据类型验证

In [ ]:
import re

_str = 'vector<string>'
str2 = 'vector<Sequence<float, 4>>'
pattern = re.compile('([1-9]\d*|string)')
type_pattern = 'Sequence|vector'
print(pattern.findall(_str))
print(pattern.match(str2))
print(re.match('vector<[a-z]*>', _str))
print(re.match('vector<(Sequence|vector)<[a-z]*, [1-9]\d*>>$', str2))
print(os.cpu_count())
re.match('_size=[1-9]\d*', '_size=1')
re.match('_t=[s|v]', '_t=v')

> 执行脚本

In [ ]:
%run main.py --f client --csv

#### 验证`require lua`

In [ ]:
import subprocess

_dir = 'Table-client/'
for file in os.listdir(_dir):
    func = 'optimizer(\"{0}\", \"{1}\")'.format(_dir + file[:-4], file[:-4])
    try:
        result = subprocess.check_output(['lua53', '-l', 'DataTableOptimizer', '-e', func])
    except Exception as e :
        print(file)

In [ ]:
import subprocess
import traceback


_dir = 'table-server'
# _dir = 'table-client'
for file in os.listdir('./' + _dir):
    func = '_load("{0}")'.format(_dir + '.' + file[:-4])
    try:
        # print(func)
        result = subprocess.check_output(['lua', '-l', 'test', '-e', func])
    except Exception as e :
        print(file)
        # pass
        # traceback.print_exc()
    # result = subprocess.check_output(['lua', '-l', 'test', '-e', func])
    

#### 自定义压缩`lua`

In [ ]:
import re
import sys
from numbers import Number
from slpp import slpp as lua
import six
import json


tab = ''
newline = ''
depth = 0

def __encode(obj):
    s = ''
    global depth
    global newline
    global tab
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif six.PY2 and isinstance(obj, unicode):
        s += '"%s"' % obj.encode('utf-8').replace(r'"', r'\"')
    elif six.PY3 and isinstance(obj, bytes):
        s += '"{}"'.format(''.join(r'\x{:02x}'.format(c) for c in obj))
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        # pass
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        depth += 1
        s += "{"
        key_list = ['%s' for k in obj.keys()]
        contents = [(key + '%s') % (k, __encode(v)) for (k, v), key in zip(obj.items(), key_list)]
        s += (',%s' % newline).join(contents)
        depth -= 1
        s += "}"
    return s

def encode(obj):
    s = ''
    if isinstance(obj, str):
        s += '"%s"' % obj.replace(r'"', r'\"')
    elif isinstance(obj, bool):
        s += str(obj).lower()
    elif obj is None:
        s += 'nil'
    elif isinstance(obj, Number):
        s += str(obj)
    elif isinstance(obj, dict):
        s += "{"
        contents = [encode(v) for _, v in obj.items()]
        s += ','.join(contents)
        s += "}"
    return s


def compress_lua(obj, name):
    s = 'local {} = {\n'.format(name)
    for _, value in obj.items():
        line = '\t{'
        for k, items in value.items():
            line += encode(items) + ','
        line = line[:-1] + '},\n'
        s += line

    # define default table
    s += '}\n\nlocal __default_table = {'
    for key, _ in indices.items():
        s += key + ','
    s = s[:-1] + '}\n'

    # add postfix

    return s

    
with open('./Table-client/AwardPackTable.lua', 'r') as f:
    table_name = 'AwardPackTable'
    text = f.read()
    obj = lua.decode(text)
    res = compress_lua(obj)
    res = 'local {} = '.format(table_name) + res
    # res = __encode(obj)
    with open('test.lua', 'w', encoding='utf-8') as w:
        w.write(res)
        w.write("\ndo\n")
        w.write("\tlocal base = {__index = __default_table, __newindex = function() error(\"Attempt to modify read-only table\") end}\n")
        w.write("\tfor k, v in pairs(%s) do\n" % (table_name))
        w.write("\t\tsetmetatable(v, base)\n")
        w.write("\tend\n")
        w.write("\tbase.__metatable = false\n")
        w.write("end\n")
        w.write("\nreturn %s\n" % (table_name))


### `pkg`导出`c++`

In [ ]:
import re
from CppHeaderParser import CppHeader, CppParseError


class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR


def filter_include(lines):
    res = ''
    for line in lines:
        if line.strip().startswith('$'): res += line.replace('$', '')
        else: res += line
    return res


def pascal_to_snake(camel: str):
    snake = re.sub(r'(?P<key>[A-Z])', r'_\g<key>',camel)
    return snake.lower().strip('_')


def parse_cpp(name, header):
    namespaces = set()
    classes = {}
    output_namespaces = 4 * ' ' + '// {}\n'.format(name)
    output_classes = 4 * ' ' + '// classes\n'
    output_variables = 4 * ' ' + '// variables\n'
    output_methods = 4 * ' ' + '// methods\n'
    output_enums = 4 * ' ' + '// enums\n'

    # variables & methods
    for class_name in header.classes.keys():
        # print(header.classes[class_name])
        namespace = header.classes[class_name]['namespace']
        namespaces.add(namespace)
        classes[class_name] = namespace
        
        for method in header.classes[class_name]['methods']['private']:
            output_methods += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), method['name'], namespace, class_name, method['name'])

        for property in header.classes[class_name]['properties']['public']:
            output_variables += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), property['name'], namespace, class_name, property['name'])

        for property in header.classes[class_name]['properties']['private']:
            output_variables += 4 * ' ' + '{}_proxy[\"{}\"] = &{}::{}::{};\n'.format(pascal_to_snake(class_name), property['name'], namespace, class_name, property['name'])

    # namespace
    for namespace in namespaces:
        output_namespaces += 4 * ' ' + 'sol::table {} = l.create_named_table(\"{}\");\n'.format(pascal_to_snake(namespace), namespace)

    # classes
    for class_, namespace in classes.items():
        output_classes += 4 * ' ' + 'auto {}_proxy = {}.new_usertype<{}::{}>(\"{}\");\n'.format(pascal_to_snake(class_), pascal_to_snake(namespace), namespace, class_, class_)

    # enums
    for enum in header.enums:
        s = ''
        enum_str = 8 * ' ' + '\"{}\", {}::{}::{}, \n'
        namespace = enum['namespace'][:-2]
        for variable in enum['values']:
            s += enum_str.format(variable['name'], namespace, enum['name'], variable['name'])
        # print(s[:-1])
        output_enums += 4 * ' ' + '{}.new_enum(\"{}\",\n{}\n);\n'.format(pascal_to_snake(namespace), enum['name'], s[:-1])

    print(output_namespaces, output_classes, output_methods, output_variables, output_enums)


with open('.config', 'r') as f:
    lines = f.readlines()
    dir_config = lines[2].strip()
    _dir = dir_config.split('#')[1]
    for file in os.listdir(_dir):
        if file.endswith('.pkg'):
            if file != 'buff_def.pkg': continue
            with open(os.path.join(_dir, file), 'r', encoding='utf-8') as pkg:
                content = filter_include(pkg.readlines())
                try:
                    cpp_header = CppHeader(content, argType='string', encoding='utf-8')
                    parse_cpp(file, cpp_header)
                except CppParseError as e:
                    # print(e)
                    print(f'{bcolors.FAIL}' + file + f'{bcolors.RESET}')

### 全局字符串

In [ ]:
from csv_to_lua import CSVToLua as csv

csv.set_writ_flag(2)
csv.setConfig('client', is_need_key=False, is_need_index=True, is_save_string=True)
csv.csv_to_lua()

global_string = csv.get_global_string()


In [ ]:
def revert_dict(d):
    result = {}
    for k in d:
        if d[k] not in result:
            result[d[k]] = set()
        result[d[k]].add(k)
    return {k: result[k] if len(result[k]) > 1 else result[k].pop() for k in result}
def one_line(index):
    if index > LOCAL_TABLE_MAX: return REPEAT_KEY_PREFIX + '[' + str(index - LOCAL_TABLE_MAX) + ']'
    else: return REPEAT_KEY_PREFIX + str(index)

REPEAT_KEY_PREFIX = '__rt'
LOCAL_TABLE_MAX = 160
lines = []
index = 1
reversed_globel_string = revert_dict(global_string)
# print(reversed_globel_string)
for v, indices in reversed_globel_string.items():
    if isinstance(indices, set):
        lines.append('local {}={}\n'.format(one_line(index), v))
        index += 1
index = 1
for idx, s in global_string.items():
    if isinstance(reversed_globel_string[s], set): lines.append('str[{}] = {}'.format(idx, one_line(index)))
    lines.append('str[{}] = "{}"'.format(idx, s))

lines

### 替换`module`测试

In [ ]:
import re
import pandas as pd

_dir = 'E:\WorkSpace\ROGame-dev\clientproj\Assets\Scripts\Lua'
modules = {}
for path, dirs, fs in os.walk(_dir):
    for f in fs:
        if f.endswith('.lua'):
            with open(os.path.join(path, f), 'r', encoding='utf-8') as file:
                pattern = re.compile('module\("\w+(\.\w+){0,1}", package.seeall\)')
                # pattern = re.compile('^module')
                s = ''.join([l for l in file])
                result = pattern.search(s)
                module_name_pattern = re.compile('"\w+(\.\w+){0,1}"')
                if result:
                    module_name = module_name_pattern.search(result.group()).group()
                    module_name = module_name[1:-1]
                    if module_name not in modules:
                        modules[module_name] = []
                    modules[module_name].append(f)
                    

modules = formatter_data = pd.DataFrame(modules.items(), columns=['Module Name', 'Files'])
modules.to_csv('modules.csv')

In [ ]:
import pandas as pd
import re
from peel_deprecated_module import PeelDeprecatedModule as pdm
from rich.console import Console

console = Console()
pdm.peel()
modules = dict(sorted(pdm.get_modules().items()))
pattern = re.compile('(?P<key>\w+)')
unique_module = set()
for g, _ in modules.items():
    res = pattern.findall(g)
    if res and res[0] not in unique_module:
        unique_module.add(res[0])
        print('\n{} = {{}}\ndeclareGlobal("{}", {})'.format(res[0], res[0], res[0]))

    if res and len(res) == 2:
        print('{}.{} = {{}}'.format(res[0], res[1]))
# formatter_data = pd.DataFrame(modules.items(), columns=['Module Name', 'Files'])

In [ ]:
from luaparser import ast
from luaparser import astnodes

class FunctionVisitor(ast.ASTVisitor):

    def visit_Function(self, node):
            if isinstance(node.name, ast.Name):
                print(str(node.name.id))

    def visit_Call(self, node):
        if isinstance(node.func, ast.Name):
            print('call function {}, more {}'.format(node.func.id, node.args))
        # if isinstance(node.func, ast.Index): print('++{}'.format(node.func.idx.id))

    # def visit_Table(self, node):
    #     # print(node.display_name)
    #     for v in node.fields:
    #         if isinstance(v.key, ast.Name): print(v.key.id)
    #         if isinstance(v.key, ast.Number): print(v.key.n)

    def visit_Assign(self, node):
        for target, value in zip(node.targets, node.values):
            if isinstance(value, ast.AnonymousFunction ):
                print('define anonymous function {}, in line {}'.format(target.id, target.line))


with open('./script/CommonUI/Color.lua', 'r', encoding='utf-8') as f:
    tree = ast.parse(''.join([line for line in f.readlines()]))

    FunctionVisitor().visit(tree)
    print(ast.to_pretty_str(tree))
    # print(ast.to_xml_str(tree))

#### 正则表达式替换相关文本测试

In [ ]:
import re

_str = 'function IsRectTransformOverlap(r1, r2)'
func = 'IsRectTransformOverlap'
pattern = re.compile('function\s*{}\(.*\)'.format(func))
# print(pattern.search(str).group())

content = '---@module CommonUI.Color\nmodule("CommonUI.Color", package.seeall)\ndeclareGlobal("RoColor", CommonUI.Color)\n'
# print(content)
p = re.compile('module\("\w+(\.\w+){0,1}", package.seeall\)')
# print(p.search(content))

enum_pattern =  re.compile(r'\b{}\b\s*[^=<>~]=[^=].*'.format('nTASK_TRIGGER_TYPE'))
# print(enum_pattern)
l = [
    '\n',
    'TASK_TRIGGER_TYPE_STR =\n',
    '{\n'
]
res = enum_pattern.search(''.join(l))
# print(res)

pattern = re.compile(r'[^\.]\bColor\b')
print(pattern.findall('\nreturn Color(a, b, v)\n'))

> 进度条测试

In [ ]:
import requests
import concurrent.futures


def get_wiki_page_existence(wiki_page_url, timeout=10):
    response = requests.get(url=wiki_page_url, timeout=timeout)

    page_status = "unknown"
    if response.status_code == 200:
        page_status = "exists"
    elif response.status_code == 404:
        page_status = "does not exist"

    return wiki_page_url + " - " + page_status


wiki_page_urls = [
    "https://en.wikipedia.org/wiki/Ocean",
    "https://en.wikipedia.org/wiki/Island",
    "https://en.wikipedia.org/wiki/this_page_does_not_exist",
    "https://en.wikipedia.org/wiki/Shark",
]
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for url in wiki_page_urls:
        futures.append(
            executor.submit(
                get_wiki_page_existence, wiki_page_url=url, timeout=0.00001
            )
        )
    for future in concurrent.futures.as_completed(futures):
        try:
            print(future.result())
        except requests.ConnectTimeout:
            print("ConnectTimeout.")

In [ ]:
import time
import requests
import concurrent.futures


def get_wiki_page_existence(wiki_page_url, timeout=10):
    response = requests.get(url=wiki_page_url, timeout=timeout)

    page_status = "unknown"
    if response.status_code == 200:
        page_status = "exists"
    elif response.status_code == 404:
        page_status = "does not exist"

    return wiki_page_url + " - " + page_status
wiki_page_urls = ["https://en.wikipedia.org/wiki/{}".format(i) for i in range(50)]

print("Running threaded:")
threaded_start = time.time()
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = []
    for url in wiki_page_urls:
        futures.append(executor.submit(get_wiki_page_existence, wiki_page_url=url))
    for future in concurrent.futures.as_completed(futures):
        print(future.result())
print("Threaded time:", time.time() - threaded_start)

In [ ]:

from time import sleep

from rich.live import Live
from rich.panel import Panel
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn
from rich.table import Table


job_progress = Progress(
    "{task.description}",
    SpinnerColumn(),
    BarColumn(),
    TextColumn("[progress.percentage]{task.percentage:>3.0f}%"),
)
job1 = job_progress.add_task("[green]Cooking")
job2 = job_progress.add_task("[magenta]Baking", total=200)
job3 = job_progress.add_task("[cyan]Mixing", total=400)

total = sum(task.total for task in job_progress.tasks)
overall_progress = Progress()
overall_task = overall_progress.add_task("All Jobs", total=int(total))

progress_table = Table.grid()
progress_table.add_row(
    Panel.fit(
        overall_progress, title="Overall Progress", border_style="green", padding=(2, 2)
    ),
    Panel.fit(job_progress, title="[b]Jobs", border_style="red", padding=(1, 2)),
)

with Live(progress_table, refresh_per_second=10):
    while not overall_progress.finished:
        sleep(0.1)
        for job in job_progress.tasks:
            if not job.finished:
                job_progress.advance(job.id)

        completed = sum(task.completed for task in job_progress.tasks)
        overall_progress.update(overall_task, completed=completed)

### 特殊`csv`导出

In [ ]:
from csv_to_lua import CSVToLua as csv
import json
import pandas as pd

csv.setConfig(**{'for': 'server', 'index': True, 'require': 'all'})

_dir = 'E:\\WorkSpace\\W4-Fight\\config\\Table\\'
json_dir = os.path.join(_dir, 'Configs')
_file = 'BuffEffectTable.json'

def process(data):
    if data is None: return
    def extract_table(name, key):
        # header adaptation
        data = pd.read_csv(os.path.join(_dir, 'CSV', name)).drop([0])
        columns = list(set(data.columns.tolist()) - set(heads[key].values()))
        data = data.drop(columns=columns)
        data = data.dropna(axis=0, how='all')
        _heads = sorted(heads[key].items(), key = lambda item : item[0])
        data = data[[x[1] for x in _heads]]
        return data

    def process_one_table(name, data):
        file_path = './table-server/' + os.path.basename(name).replace('.csv', '.lua')

        print(file_path)
        # sort table if server
        csv.get_primary_index(_file[:-5])
        try:
            _index = csv.primary_index['key']
            if _index and not data.empty:
                _type = types[csv._extract_name(_file[:-5])][_index]['FieldTypeName']
                data[_index] = data[_index].astype(int if _type in ['int', 'uint', 'long long'] else object)
                data.sort_values(_index, inplace=True)
        except Exception as e:
            lua_raw_data = data.to_dict('index')
            table = csv.iter_csv_recursive(lua_raw_data, _file[:-5])
            try:
                # print(f'processing {self.bcolors.OK}' + name + f'{self.bcolors.RESET} ...')
                name = csv._extract_name(name)
                with open(file_path, 'w', encoding='utf-8') as w:
                    w.write(csv.compress_lua(table, _file[:-5]))
            except Exception as e:
                traceback.print_exc()
    for item in data['TableLocations']:
        name = item['ExcelPath']
        process_one_table(name, extract_table(name, data['MainTableName']))

csv._load_heads(os.listdir(json_dir))
types, heads = csv.get_heads()
# print(types['BuffEffectTable'])
with open(os.path.join(json_dir, _file), 'rb') as file:
    data = json.load(file)
    process(data)
    process(data['Children'][0] if data['Children'] else None)

### 技能表合并

#### 合并

In [23]:
import pandas as pd
# from rich import print


test = pd.read_csv('./skill-table/SkillTable.csv')
test = test.drop(['Name.1'], axis = 1)
columns = [v for v in test.columns if 'Unnamed' not in v]
test = test[columns]
test_columns = set(test.columns.tolist())
# print(test_columns)

skill_table = pd.read_csv('E:\\WorkSpace\\W4-Fight\\config\\Table\\CSV\\SkillTable.csv')
skill_table.rename(columns = {'Id' : 'SkillID'}, inplace = True)
# print(skill_table.columns)
# skill_table = set(skill_table.columns.tolist())

skill_effect_table = pd.read_csv('E:\\WorkSpace\\W4-Fight\\config\\Table\\CSV\\SkillEffectTable.csv')
skill_effect_table.rename(columns = {'SkillElementEnergy' : 'SkillEffectElementEnergy'}, inplace = True)
columns = [v for v in skill_effect_table.columns if 'Unnamed' not in v]
skill_effect_table = skill_effect_table[columns]
skill_effect_table.drop(['Name'], axis = 1, inplace = True)
# print(skill_effect_table.columns)
# skill_effect_table = set(skill_effect_table.columns.tolist())

merge = pd.merge(left = skill_effect_table, right = skill_table, on = 'SkillID', how = 'left')
merge['ScriptParameter'] = None
merge = merge[test.columns.tolist()]
merge.iloc[0] = test.iloc[0]
merge.to_csv('SkillTable.csv', index = False, encoding = 'utf-8-sig')
# print(test.columns.tolist())

from csv_to_lua import CSVToLua as csv
import json
import pandas as pd
import traceback


def load_types(file_name, indices, _types, start = 0):
    types = {}
    with open(file_name, 'rb') as file:
        data = json.load(file)
        index = 0
        for field in data['Fields']:
            if field['FieldName'] in indices:
                if field['FieldName'] == 'SkillElementEnergy' and start != 0:
                    types['SkillEffectElementEnergy'] = {'FieldTypeName' : 'vector<int>', 'Pos' : index + start}
                elif field['FieldName'] not in _types:
                    types[field['FieldName']] = {'FieldTypeName' : field['FieldTypeName'], 'Pos' : index + start}
                index += 1
    return types

skill_json_dir = 'E:\\WorkSpace\\W4-Fight\\config\\Table\\Configs\\SkillTable.json'
skill_effect_json_dir = 'E:\\WorkSpace\\W4-Fight\\config\\Table\\Configs\\SkillEffectTable.json'

types = {}

for i in range(len(merge.columns)):
    types[merge.columns.tolist()[i]] = {'FieldTypeName' : 'string', 'Pos' : i}

with open(skill_json_dir, 'rb') as file:
        data = json.load(file)
        for field in data['Fields']:
            if field['FieldName'] in types:
                types[field['FieldName']]['FieldTypeName'] = field['FieldTypeName']

with open(skill_effect_json_dir, 'rb') as file:
        data = json.load(file)
        for field in data['Fields'][1:]:
            if field['FieldName'] in types:
                if field['FieldName'] == 'SkillElementEnergy':
                    types['SkillEffectElementEnergy']['FieldTypeName'] = field['FieldTypeName']
                else:
                    types[field['FieldName']]['FieldTypeName'] = field['FieldTypeName']

types['ScriptParameter']['FieldTypeName'] = 'string'

def process_one_table(name, data, types):
    lua_raw_data = data.to_dict('index')
        
    table = csv.iter_csv_recursive(lua_raw_data, '', types = types)
    with open(name + '.lua', 'w', encoding='utf-8') as w:
        w.write(csv.compress_lua(table, 'SkillTable', types))

csv.setConfig(**{})
# process_one_table('SkillTable', merge.drop([0]), types)

#### 类型处理

In [24]:
def _str(t):
    res = ''
    if t == int: res =  'int'
    elif t == float: res = 'float'
    elif t == str: res = 'string'
    elif t == bool: res = 'bool'
    return res

# print(merge.iloc[0])
declaration = []
for k, v in types.items():
    desc = merge.iloc[0][k]
    # desc = desc[0:8] + '...'
    _match = csv.regex_type(v['FieldTypeName'])
    declaration.append('\n[LabelText("{}"), LabelWidth({})]'.format(desc, 80))
    if isinstance(_match, tuple):
        if _match[0] == 's':
            declaration.append('public {}[] {};\n'.format(_str(_match[1][2]), k))
        elif _match[0] == 'v':
            declaration.append('public List<{}> {};\n'.format(_str(_match[1]), k))
        elif _match[0] == 'vv':
            declaration.append('public List<List<{}>> {};\n'.format(_str(_match[1]), k))
        elif _match[0] == 'vs':
            declaration.append('public List<{} []> {};\n'.format(_str(_match[1][2]), k))
    else:
        declaration.append('public {} {};\n'.format(v['FieldTypeName'], k))
print('\n'.join(declaration))

[LabelText("唯一技能ID"), LabelWidth(80)]
public int Id;


[LabelText("技能标识ID，通过buff换技能用（不同等级ID相同）"), LabelWidth(80)]
public int SkillUuid;


[LabelText("等级"), LabelWidth(80)]
public int Level;


[LabelText("技能名字（策划用备注）"), LabelWidth(80)]
public string NameDesign;


[LabelText("描述"), LabelWidth(80)]
public string Desc;


[LabelText("技能名字"), LabelWidth(80)]
public string Name;


[LabelText("是否沿用端游翻译（0否 1是）"), LabelWidth(80)]
public string IsIP;


[LabelText("技能目标类型\r\n（被动-1 ， 0所有，1自己，2友方，3队伍，4敌人，5死亡的友方, 
6自己的宠物, 7自己的最终召唤者）"), LabelWidth(80)]
public int TargetType;


[LabelText("技能目标范围类型\r\n（-1.被动 0.目标 1.方向 2.坐标 3.自己）"), LabelWidth(80)]
public int SkillTargetRangeType;


[LabelText("技能射程类型\r\n（0.近战  1.远程  2.随释放时距离目标的距离而定（a.>=4米远程  
b.<4米近战（距离global表配置）））"), LabelWidth(80)]
public int SkillRangeType;


[LabelText("技能仇恨类型\r\n（被动-1 ，1.伤害 2.支援 3.嘲讽类 4.伤害+支援类  ）"), 
LabelWidth(80)]
public int SkillHatedType;


[LabelText("技能类型（0.普通技能  1.偷窃类技能  2.怪物互击  3.位移类技能 4.解控类技能 
7.骑乘类技能）"), LabelWidth(80)]
public int SkillType;


[LabelText("技能伤害类型\r\n（被动-1 ，0普攻   1物理，2魔法，3混合）"), LabelWidth(80)]
public int SkillDamType;


[LabelText("是否被ASPD影响\r\n（被动-1 ，0.不被影响  1.被影响）"), LabelWidth(80)]
public int IsAffectedASPD;


[LabelText("ASPD影响百分比"), LabelWidth(80)]
public float ASPDPercent;


[LabelText("技能产生的共CD是否受到ASPD加成（被动技能填-1 ，0.不被影响  1.被影响）"), 
LabelWidth(80)]
public bool IsAffectedASPDExtra;


[LabelText("是否AOE技能\r\n（0.否    1.是）"), LabelWidth(80)]
public bool IsAoe;


[LabelText("技能状态被打断配置（0不可被击退 1可以被击退但不打断 
2可以被击退并且打断技能状态）"), LabelWidth(80)]
public int PushInterupt;


[LabelText("技能属性（被动技能填-1 ，）"), LabelWidth(80)]
public int SkillAttr;


[LabelText("是否可以击飞"), LabelWidth(80)]
public bool IsKnockBack;


[LabelText("技能失效范围（填0时默认为20）"), LabelWidth(80)]
public int Length;


[LabelText("是否受射程属性影响（0.不受  1.受）"), LabelWidth(80)]
public bool IsRangeAttrImpact;


[LabelText("是否和武器比较射程（0.不比较 1.比较）"), LabelWidth(80)]
public bool IsRangeWeaponAttrImpact;


[LabelText("武器限制（不填无限制，填则走EquipWeaponTable定义的值，可多填）"), LabelWidth(80)]
public List<int> WeaponTypeLimit;


[LabelText("副手武器限制（不填无限制，填则走EquipWeaponTable定义的值，可多填）"), 
LabelWidth(80)]
public List<int> SecondWeaponTypeLimit;


[LabelText("技能同时存在个数上限（无则不填）"), LabelWidth(80)]
public string Limits;


[LabelText("是否隐藏CD（0不隐藏，1隐藏）"), LabelWidth(80)]
public bool IsDisableCD;


[LabelText("额外的技能ID数组(每个等级一个ID)"), LabelWidth(80)]
public List<int> EffectExIDs;


[LabelText("技能位置"), LabelWidth(80)]
public int[] SkillPanelPos;


[LabelText("特殊前置条件（0任务）"), LabelWidth(80)]
public List<List<int>> SpePreSkillRequired;


[LabelText("释放后是否自动普攻（0释放后接自动普攻，1释放后不接自动普攻）"), LabelWidth(80)]
public bool LinkAutoAtack;


[LabelText("释放技能是否改变目标颜色（0为不改颜色，1为改颜色）"), LabelWidth(80)]
public bool IsChangeColor;


[LabelText("释放技能是否改变目标颜色（填写模板名则改变，不填写则不变）"), LabelWidth(80)]
public string ChangeColor;


[LabelText("可强制打断的技能ID（可配置多个）"), LabelWidth(80)]
public List<int> InteruptSkill;


[LabelText("是否使用箭矢及消耗数量"), LabelWidth(80)]
public int[] IsUseArrow;


[LabelText("额外的图标（当额外技能是独立技能时有用）"), LabelWidth(80)]
public string IconEx;


[LabelText("指定场景ID不可释放"), LabelWidth(80)]
public List<int> LimitScene;


[LabelText("指定场景类型不可释放"), LabelWidth(80)]
public List<int> LimitSceneType;


[LabelText("是否需要显示施法范围"), LabelWidth(80)]
public bool NeedEffectRange;


[LabelText("额外技能是否为独立的技能"), LabelWidth(80)]
public bool IsEffectExStandalone;


[LabelText("是否为被动技能"), LabelWidth(80)]
public bool IsPassive;


[LabelText("自动战斗释放优先级（默认为0，数字越大优先级越高）"), LabelWidth(80)]
public int AutoBattlePRI;


[LabelText("技能能量（填SkillEffectTable表中对应技能的不同等级的最大值，没有则不填）"), 
LabelWidth(80)]
public List<int> SkillEffectElementEnergy;


[LabelText("能量不足时的提示（只有能量类技能才会显示，对应ChineseTable字段）"), 
LabelWidth(80)]
public string EnergyTips;


[LabelText("动作文件夹（被动技能填-1 ，0通用，1职业）"), LabelWidth(80)]
public int AnimationType;


[LabelText("动作播放顺序（-1.随机   0.按序  >0.按照配置顺序的序号播放）"), LabelWidth(80)]
public int SingingAnimation;


[LabelText("前置技能和等级(id=lv,id=lv)"), LabelWidth(80)]
public List<int []> PreSkillRequir